In [ ]:
import numpy as np
import torch
import torch.nn
import torch.optim
from torch.utils.data import DataLoader
from abc import abstractmethod
from collections import defaultdict
from functools import lru_cache
from itertools import count
from typing import List, Dict
from typing import Tuple, Any
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from torch.nn import MSELoss, ReLU, L1Loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
from base_module import MModule
from data import MDataset, Graph, load_graphs
from importlib import reload
from config import Config
import config
from data import MDataset, Graph, load_graphs, save_dataset_pkl, load_dataset_pkl, save_scalers_pkl, load_scalers_pkl
import data
from base_module import MModule, nested_detach
import base_module
from executor import single_train_loop, grid_search_loop
import executor
from objects import ModelType
import objects
from metric import MetricUtil
import metric
from logger import init_logging
import logger
reload(config)
reload(data)
reload(base_module)
reload(executor)
reload(objects)
reload(metric)
reload(logger)
init_logging()

In [ ]:
dataset_environment_str = "P4_CPUALL"
normalizer_cls = StandardScaler  # MinMaxScaler
dummy = False
op_feature_scaler, y_scaler = None, None
eval_graphs = None

In [ ]:
train_configs = {
    ModelType.MLP.name: Config.from_dict({
        "model": "MLP",
        "all_seed": 42,
        "dataset_environment_str": dataset_environment_str,
        "dataset_normalization": "Standard",
        "dataset_params": {
            "duration_summed": False,
        },
        "dataset_dummy": False,
        "batch_size": 32,
        "eval_steps": 10000,
        "learning_rate": 1e-4,
        "epochs": 10,
        "optimizer": "Adam",
        "meta_configs": {
            "learning_rate": 0.005,
            "meta_learning_rate": 0.001,
            "meta_train_steps": 1000,
            "meta_task_per_step": 8,
            "meta_fast_adaption_step": 5,
            "meta_dataset_train_environment_strs": [dataset_environment_str],
            "meta_dataset_eval_environment_strs": [dataset_environment_str],
        },
    }),
    ModelType.PerfNet.name: Config.from_dict({
        "model": "PerfNet",
        "dataset_environment_str": dataset_environment_str,
        "dataset_normalization": "Standard",
        "all_seed": 42,
        "dataset_params": {
            "duration_summed": False,
        },
        "dataset_dummy": True,
        "batch_size": 32,
        "eval_steps": 10000,
        "learning_rate": 1e-4,
        "epochs": 10,
        "optimizer": "Adam",
        "meta_configs": {
            "learning_rate": 0.005,
            "meta_learning_rate": 0.001,
            "meta_train_steps": 1000,
            "meta_task_per_step": 8,
            "meta_fast_adaption_step": 5,
            "meta_dataset_train_environment_strs": [dataset_environment_str],
            "meta_dataset_eval_environment_strs": [dataset_environment_str],
        },
    })
}

model_type = ModelType.MLP
conf = train_configs[model_type.name]

In [ ]:

def init_dataset(graphs: List[Graph]) -> MDataset:
    op_X, op_Y = list(), list()
    data_idx_to_graph = dict()
    counter = iter(count())
    op_feature_len = 0

    def node_features(g: Graph) -> Tuple[
            List[Dict], List[Dict]]:
        X, Y = list(), list()
        for i, node in enumerate(g.nodes):
            x_op_feature = node.op.to_feature_array("complex")
            x = {
                "x_op_feature": x_op_feature
            }
            # node_durations = (node.duration,node.gap)
            node_durations = (node.duration+node.gap,)
            x["x_id"] = i
            x["x_graph_id"] = g.ID
            y = {"y_node_durations": node_durations,
                 "y_id": i, "y_graph_id": g.ID}
            X.append(x)
            Y.append(y)
        return X, Y

    for graph in graphs:
        X, Y = node_features(graph)
        for x in X:
            # if len(x['x_op_feature'])!=37:
            #     print(x['x_graph_id'], len(x['x_op_feature']))
            op_feature_len = max(op_feature_len, len(x["x_op_feature"]))
        op_X.extend(X)
        op_Y.extend(Y)
        for i in range(len(X)):
            data_idx_to_graph[next(counter)] = graph
    for x in op_X:
        v = x["x_op_feature"]
        x["x_op_feature"] = np.pad(v, (0, op_feature_len - v.size))

    dataset = MDataset(op_X, op_Y)
    return dataset

In [ ]:
def get_scalers(ds):
    scaler_cls = normalizer_cls
    op_feature_array = list()
    y_array = list()

    for data in ds:
        feature, label = data
        op_feature_array.append(feature["x_op_feature"])
        y_array.append(label["y_node_durations"])

    op_feature_array = np.array(op_feature_array)
    y_array = np.array(y_array)

    op_feature_scaler = scaler_cls()
    op_feature_scaler.fit(op_feature_array)

    y_scaler = scaler_cls()
    y_scaler.fit(y_array)
    return op_feature_scaler, y_scaler

In [ ]:

def preprocess_dataset(ds: MDataset) -> MDataset:
    op_feature_array = list()
    y_array = list()

    for data in ds:
        feature, label = data
        op_feature_array.append(feature["x_op_feature"])
        y_array.append(label["y_node_durations"])

    op_feature_array = np.array(op_feature_array, dtype=np.float32)
    y_array = np.array(y_array, dtype=np.float32)

    op_feature_array = op_feature_scaler.transform(op_feature_array)
    y_array = y_scaler.transform(y_array)

    processed_features = list()
    processed_labels = list()
    for i, data in enumerate(ds):
        feature, label = data
        processed_features.append({
            "x_id": feature["x_id"],
            "x_graph_id": feature["x_graph_id"],
            # 运行时再传到cuda那边
            # "x_op_feature": torch.Tensor(op_feature_array[i]).to(device=self.conf.device)
            "x_op_feature": torch.Tensor(op_feature_array[i])
        })
        processed_labels.append({
            "y_id": label["y_id"],
            "y_graph_id": label["y_graph_id"],
            # "y_node_durations": torch.Tensor(y_array[i]).to(device=self.conf.device)
            "y_node_durations": torch.Tensor(y_array[i])
        })

    ds = MDataset(processed_features, processed_labels)
    return ds

In [ ]:
# save_dataset_pkl(preprocessed_train_ds, conf.dataset_environment, "OpBased", 'train',
#                          conf.dataset_normalization)
# save_dataset_pkl(preprocessed_eval_ds, conf.dataset_environment, "OpBased", 'eval',
#                          conf.dataset_normalization)
# save_scalers_pkl(scalers, conf.dataset_environment, "OpBased", 'train',
#                          conf.dataset_normalization)

In [ ]:
# preprocessed_train_ds = load_dataset_pkl(conf.dataset_environment, "OpBased", 'train',
#                                          conf.dataset_normalization)
# preprocessed_eval_ds = load_dataset_pkl(conf.dataset_environment, "OpBased", 'eval',
#                                         conf.dataset_normalization)
# scalers = load_scalers_pkl(conf.dataset_environment, "OpBased", 'train',
#                            conf.dataset_normalization)

In [ ]:
def compute_evaluate_metrics(input_batches, output_batches, eval_loss) -> Dict[str, float]:
    batches_len = len(input_batches)

    def compute_op_durations(_logits):
        transformed: np.ndarray = y_scaler.inverse_transform(_logits)
        durations = transformed.sum(axis=1)
        return durations

    graph_id_to_duration_pred = defaultdict(int)
    for idx in range(batches_len):
        inputs = input_batches[idx]
        logits = output_batches[idx]
        logits = nested_detach(logits)
        logits = logits.cpu().numpy()
        graph_ids = inputs["x_graph_id"]
        op_durations = compute_op_durations(logits)
        for i, graph_id in enumerate(graph_ids):
            op_duration = op_durations[i].item()
            graph_id_to_duration_pred[graph_id] += op_duration
    duration_metrics = MetricUtil.compute_duration_metrics(
        eval_graphs, graph_id_to_duration_pred)
    return {"eval_loss": eval_loss, **duration_metrics}

In [ ]:
def compute_durations(input_batches, output_batches, scalers, eval_graphs) -> Dict[str, float]:

    batches_len = len(input_batches)

    def compute_op_durations(_logits):
        transformed: np.ndarray = y_scaler.inverse_transform(_logits)
        durations = transformed.sum(axis=1)
        return durations

    graph_id_to_duration_pred = defaultdict(int)
    for idx in range(batches_len):
        inputs = input_batches[idx]
        logits = output_batches[idx]
        logits = nested_detach(logits)
        logits = logits.cpu().numpy()
        graph_ids = inputs["x_graph_id"]
        op_durations = compute_op_durations(logits)
        for i, graph_id in enumerate(graph_ids):
            op_duration = op_durations[i].item()
            graph_id_to_duration_pred[graph_id] += op_duration

    y_hat, y = list(), list()
    for graph in eval_graphs:
        y_hat.append(graph_id_to_duration_pred[graph.ID])
        y.append(graph.graph_duration)
    return np.array(y_hat), np.array(y)

In [ ]:

def to_device(conf, features, labels):
    features['x_op_feature'] = features["x_op_feature"].to(device='cuda')
    labels['y_node_durations'] = labels['y_node_durations'].to(
        device='cuda')
    return features, labels

In [ ]:
class MLPModel(MModule):

    @staticmethod
    def dimension_len(t):
        return t[-1] - t[0]

    @staticmethod
    def grid_search_model_params() -> Dict[str, List[Any]]:
        return {
            "learning_rate": [1e-3, 1e-4, 1e-5],
            'batch_size': [16, 32, 64],
            'epochs': [5, 10, 20, 30],
            'optimizer': ['Adam', 'SGD'],
        }
        # return {}

    def __init__(self, input_dimension, output_dimension, **kwargs):
        super().__init__(**kwargs)
        self.input = torch.nn.Linear(input_dimension, 256)
        self.relu1 = ReLU()
        self.dense1 = torch.nn.Linear(256, 128)
        self.relu2 = ReLU()
        self.dense2 = torch.nn.Linear(128, 64)
        self.relu3 = ReLU()
        self.output = torch.nn.Linear(64, output_dimension)
        self.loss_fn = L1Loss()

    def forward(self, X):
        X = X["x_op_feature"]
        X = self.input(X)
        X = self.relu1(X)
        X = self.dense1(X)
        X = self.relu2(X)
        X = self.dense2(X)
        X = self.relu3(X)
        Y = self.output(X)
        return Y

    def compute_loss(self, outputs, Y):
        node_durations = Y["y_node_durations"]
        loss = self.loss_fn(outputs, node_durations)
        return loss


def init_MLP_model() -> MModule | Any:
    # sample_preprocessed_ds = preprocessed_train_ds
    # sample_x_dict = sample_preprocessed_ds.features[0]
    # sample_y_dict = sample_preprocessed_ds.labels[0]
    # return MLPModel(input_dimension=len(sample_x_dict["x_op_feature"]),
    #                 output_dimension=len(sample_y_dict["y_node_durations"]))
    return MLPModel(66, 2)

In [ ]:

class PerfNetModel(MModule):
    @staticmethod
    def dimension_len(t):
        return t[-1] - t[0]

    @staticmethod
    def grid_search_model_params() -> Dict[str, List[Any]]:
        return {
            "learning_rate": [1e-3, 1e-4, 1e-5],
            'batch_size': [16, 32, 64],
            'epochs': [5, 10, 20, 30],
            'optimizer': ['Adam', 'SGD'],
        }
        # return {}

    def __init__(self, output_dimension, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = torch.nn.LazyConv1d(
            out_channels=32, kernel_size=3, bias=True, padding_mode='zeros')
        self.conv2 = torch.nn.LazyConv1d(
            out_channels=128, kernel_size=2, bias=True, padding_mode='zeros')
        self.flatten = torch.nn.Flatten()
        self.dense1 = torch.nn.LazyLinear(32)
        self.relu1 = ReLU()
        self.dense2 = torch.nn.LazyLinear(64)
        self.relu2 = ReLU()
        self.dense3 = torch.nn.LazyLinear(128)
        self.relu3 = ReLU()
        self.dense4 = torch.nn.LazyLinear(256)
        self.relu4 = ReLU()
        self.dropout = torch.nn.Dropout(p=0.3)
        self.output = torch.nn.LazyLinear(output_dimension)
        self.loss_fn = L1Loss()

    def forward(self, X):
        X = X["x_op_feature"]
        X = torch.unsqueeze(X, dim=1)
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.flatten(X)
        X = self.dense1(X)
        X = self.relu1(X)
        X = self.dense2(X)
        X = self.relu2(X)
        X = self.dense3(X)
        X = self.relu3(X)
        X = self.dense4(X)
        X = self.relu4(X)
        X = self.dropout(X)
        Y = self.output(X)
        return Y

    def compute_loss(self, outputs, Y):
        node_durations = Y["y_node_durations"]
        loss = self.loss_fn(outputs, node_durations)
        return loss


def init_PerfNet_model() -> MModule | Any:
    # sample_y_dict = preprocessed_train_ds.labels[0]
    # print(len(sample_y_dict["y_node_durations"]))
    # return PerfNetModel(output_dimension=len(sample_y_dict["y_node_durations"]))
    return PerfNetModel(output_dimension=1)

In [ ]:
# eval_process
eval_size = 200_000
batch_size = 32
method_prefix = 'OpBased'
models = {
    'T4_CPUALL': '/root/guohao/DLT-perf-model/notebooks/ckpts/op_based/T4_CPUALL/PerfNet/single_train2024-01-09_12-45-50/ckpt_835000.pth',
    'P4_CPUALL': '/root/guohao/DLT-perf-model/notebooks/ckpts/op_based/P4_CPUALL/PerfNet/single_train2024-01-09_14-06-21/ckpt_305000.pth',
    'RTX2080Ti_CPUALL': '/root/guohao/DLT-perf-model/notebooks/ckpts/op_based/RTX2080Ti_CPUALL/PerfNet/single_train2024-01-09_15-00-18/ckpt_930000.pth',
    'RTX3080Ti_CPUALL': '/root/guohao/DLT-perf-model/notebooks/ckpts/op_based/RTX3080Ti_CPUALL/PerfNet/single_train2024-01-09_16-24-24/ckpt_815000.pth',
}
res = []
for dataset_environment_str, model_ckpt in models.items():
    eval_graphs = load_graphs(dataset_environment_str,
                              train_or_eval="eval",
                              use_dummy=False,
                              max_row=eval_size)

    eval_ds = init_dataset(eval_graphs)
    scalers = load_scalers_pkl(dataset_environment_str, method_prefix, 'train',
                               'Standard')
    op_feature_scaler, y_scaler = scalers
    preprocessed_eval_ds = preprocess_dataset(eval_ds)
    model = torch.load(model_ckpt)
    model.eval()
    ds = preprocessed_eval_ds
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False)
    input_batchs, output_batchs = list(), list()
    for data in dl:

        features, labels = data
        features, labels = to_device("cuda", features, labels)
        with torch.no_grad():
            outputs = model(features)
        input_batchs.append(features)
        output_batchs.append(outputs)
    y_hat, y = compute_durations(
        input_batchs, output_batchs, scalers, eval_graphs)
    rand_y, rand_y_hat = [], []
    models_y, models_y_hat = [], []
    for i in range(len(eval_graphs)):
        graph = eval_graphs[i]
        if 'rand' in graph.ID:
            rand_y.append(y[i])
            rand_y_hat.append(y_hat[i])
        else:
            models_y.append(y[i])
            models_y_hat.append(y_hat[i])
    res.append(
        {
            'dataset': dataset_environment_str,
            'mre': MetricUtil.mre(y, y_hat),
            'rmse': MetricUtil.rmse(y, y_hat),
            'rand_mre': MetricUtil.mre(rand_y, rand_y_hat),
            'rand_rmse': MetricUtil.rmse(rand_y, rand_y_hat),
            'models_mre': MetricUtil.mre(models_y, models_y_hat),
            'models_rmse': MetricUtil.rmse(models_y, models_y_hat),
        }
    )

In [ ]:
import pandas as pd
import os
df = pd.DataFrame(res)
df.to_csv(os.path.join('/root/guohao/DLT-perf-model/notebooks/exp/total_compare',
          f'OpBased.result.csv'), index=False)

In [ ]:
# envs = ['T4_CPUALL', 'P4_CPUALL', 'RTX2080Ti_CPUALL', 'RTX3080Ti_CPUALL',
#         'T4_CPU100', 'P4_CPU100', 'RTX2080Ti_CPU100', 'T4_CPU80', 'P4_CPU80']
envs = ['T4_CPUALL', 'P4_CPUALL', 'RTX2080Ti_CPUALL', 'RTX3080Ti_CPUALL']
for env in envs:
    train_configs = {
        ModelType.MLP.name: Config.from_dict({
            "model": "MLP",
            "all_seed": 42,
            "dataset_environment_str": env,
            "dataset_normalization": "Standard",
            "dataset_params": {
                "duration_summed": False,
            },
            "dataset_dummy": False,
            "batch_size": 32,
            "eval_steps": 10000,
            "learning_rate": 1e-4,
            "epochs": 10,
            "optimizer": "Adam",
            "meta_configs": {
                "learning_rate": 0.005,
                "meta_learning_rate": 0.001,
                "meta_train_steps": 1000,
                "meta_task_per_step": 8,
                "meta_fast_adaption_step": 5,
                "meta_dataset_train_environment_strs": [dataset_environment_str],
                "meta_dataset_eval_environment_strs": [dataset_environment_str],
            },
        }),
        ModelType.PerfNet.name: Config.from_dict({
            "model": "PerfNet",
            "dataset_environment_str": env,
            "dataset_normalization": "Standard",
            "all_seed": 42,
            "dataset_params": {
                "duration_summed": False,
            },
            "dataset_dummy": True,
            "batch_size": 32,
            "eval_steps": 5000,
            "learning_rate": 1e-4,
            "epochs": 30,
            "optimizer": "Adam",
            "meta_configs": {
                "learning_rate": 0.005,
                "meta_learning_rate": 0.001,
                "meta_train_steps": 1000,
                "meta_task_per_step": 8,
                "meta_fast_adaption_step": 5,
                "meta_dataset_train_environment_strs": [dataset_environment_str],
                "meta_dataset_eval_environment_strs": [dataset_environment_str],
            },
        })
    }
    # eval_graphs = load_graphs(env,
    #                           train_or_eval="eval",
    #                           use_dummy=dummy,
    #                           max_row=200_000)
    train_graphs = load_graphs(env,
                               train_or_eval="train",
                               use_dummy=dummy,
                               max_row=1000_000)
    train_ds = init_dataset(train_graphs)
    eval_ds = init_dataset(eval_graphs)
    scalers = get_scalers(train_ds)
    save_scalers_pkl(scalers, env, "OpBased", 'train',
                     conf.dataset_normalization)
    op_feature_scaler, y_scaler = scalers

    preprocessed_train_ds = preprocess_dataset(train_ds)
    preprocessed_eval_ds = preprocess_dataset(eval_ds)

    model_type = ModelType.PerfNet
    conf = train_configs[model_type.name]
    init_model_funcs = {
        ModelType.MLP.name: init_MLP_model,
        ModelType.PerfNet.name: init_PerfNet_model,
    }

    model_type = ModelType.PerfNet
    init_model = init_model_funcs[model_type.name]

    model = init_model()
    model = model.to(conf.device)
    single_train_loop(model_type, conf, preprocessed_train_ds,
                      preprocessed_eval_ds, model, compute_evaluate_metrics, to_device, suffix="op_based")